<a href="https://colab.research.google.com/github/ccsed/ProgettoLabAI/blob/main/Funzioni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collego google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importo librerie

In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 50.1 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import rasterio
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
import albumentations as A
from PIL import Image
import matplotlib.pyplot as plt
import random
import skimage.io as io
from rasterio.plot import show

GPU

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Funzione per dividere il dataset in training, validation e test set

In [4]:
def split_dataset(image_dir, drive_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
  images = sorted(os.listdir(image_dir))

  combined = [(img, img) for img in images]
  random.seed(seed)
  random.shuffle(combined)

  train_split = int(len(combined) * train_ratio)
  val_split = int(len(combined) * (train_ratio + val_ratio))
  train_data = combined[:train_split]
  val_data = combined[train_split:val_split]
  test_data = combined[val_split:]

  with open(os.path.join(drive_dir, 'train.txt'), 'w') as f:
    for img, lbl in train_data:
      s = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity/' + img
      f.write(f"{s} {s}\n")

  with open(os.path.join(drive_dir, 'val.txt'), 'w') as f:
    for img, lbl in val_data:
      s = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity/' + img
      f.write(f"{s} {s}\n")


  with open(os.path.join(drive_dir, 'test.txt'), 'w') as f:
    for img, lbl in test_data:
      s = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity/' + img
      f.write(f"{s} {s}\n")


Definisco alcune directory

In [6]:
drive_dir = '/content/drive/MyDrive'
image_dir = '/content/drive/MyDrive/Data/SN6_buildings_AOI_11_Rotterdam_train/train/AOI_11_Rotterdam/SAR-Intensity'
train_path = '/content/drive/MyDrive/train.txt'
val_path = '/content/drive/MyDrive/val.txt'
test_path = '/content/drive/MyDrive/test.txt'

Faccio split del dataset

In [7]:
split_dataset(image_dir, drive_dir)